In [29]:
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import torch
import os
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import numpy as np
import pandas as pd
image_dir = "train_images/"
save_dir = "train_images_cropped/"

class WhaleDataset_eval(data.Dataset):
    def __init__(self, csv):
        
        self.df = pd.read_csv(csv)
        self.resize = transforms.Compose([
                                         transforms.ToPILImage(),
                                         transforms.Resize((256,256)),
            transforms.ToTensor(),
                                         transforms.Normalize(
                                            mean=[0.485, 0.456, 0.406], 
                                            std=[0.229, 0.224, 0.225]
                                        )                               
        ])
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):         
        img = torchvision.io.read_image("train_images/" + self.df["image"].iloc[index]).float()
       
        if img.shape[0] == 1:
            img = torch.cat((img,img,img))
        
        img = self.resize(img)
       
         
        
        return {
            'image': img,
             "path" : self.df["image"].iloc[index]
  
        }
dataset = WhaleDataset_eval("train_corrected.csv")
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 6, 'pin_memory': True} if use_cuda else {}

train_loader = data.DataLoader(dataset=dataset,
                                batch_size=256,
                              
                                
                                **kwargs)

img_count = 0
for (batch_idx, batch) in tqdm(enumerate(train_loader)):
    images = batch["image"].to(device) 
    paths = batch["path"]
    for i in range(len(images)):
        torchvision.utils.save_image(images[i],"train_images_cropped/" + paths[i])
        img_count +=1
    

200it [22:50,  6.85s/it]


In [32]:
print(len(os.listdir("train_images_cropped/")))

51033
